Environment Setup

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import logging
from scipy.ndimage import zoom
from torch.utils.data import DataLoader, TensorDataset
from PIL import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
device

device(type='cpu')

Model Parameter

In [22]:
input_size = 196  # 14x14
hidden_size = 10 
num_classes = 10
num_epochs = 10
batch_size = 256
learning_rate = 0.001

Network Arch

In [23]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes) 

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

Prepare Dataset

In [24]:
def resize_images(images):
    return np.array([zoom(image[0], (0.5, 0.5)) for image in images])

def prepare_datasets():
    print("Prepare dataset...")
    train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True)
    test_dataset = torchvision.datasets.MNIST(root='./data', train=False)

    x_train = resize_images(train_dataset)
    x_test = resize_images(test_dataset)

    x_train = torch.tensor(x_train.reshape(-1, 14*14).astype('float32') / 255)
    y_train = torch.tensor([label for _, label in train_dataset], dtype=torch.long)

    x_test = torch.tensor(x_test.reshape(-1, 14*14).astype('float32') / 255)
    y_test = torch.tensor([label for _, label in test_dataset], dtype=torch.long)

    print("✅ Datasets prepared successfully")

    return x_train, y_train, x_test, y_test

Dataset Loader

In [25]:
def create_data_loaders(x_train, y_train, x_test, y_test):
    print("Create loaders...")

    train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

    print("✅ Loaders created!")

    return train_loader, test_loader

Train the model

In [26]:
def train_model(train_loader):
    print("Train model...")

    model = NeuralNet(input_size, hidden_size, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device).reshape(-1, 14*14)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i + 1) % 100 == 0:
                print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    print("✅ Model trained successfully")
    return model

Testing

In [27]:
def test_model(model, test_loader):
    print("Test model...")
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        for images, labels in test_loader:
            images = images.to(device).reshape(-1, 14*14)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

        acc = 100.0 * n_correct / n_samples
        print(f'Accuracy of the network on the 10000 test images: {acc} %')

In [28]:
def execution():
    # Prepare training and testing datasets
    x_train, y_train, x_test, y_test = prepare_datasets()

    train_loader, test_loader = create_data_loaders(x_train, y_train, x_test, y_test)

    model = train_model(train_loader)
 
    test_model(model, test_loader)
    
    return model

model = execution()

Prepare dataset...
✅ Datasets prepared successfully
Create loaders...
✅ Loaders created!
Train model...
Epoch [1/10], Step [100/235], Loss: 1.8380
Epoch [1/10], Step [200/235], Loss: 1.2667
Epoch [2/10], Step [100/235], Loss: 0.7495
Epoch [2/10], Step [200/235], Loss: 0.6124
Epoch [3/10], Step [100/235], Loss: 0.4208
Epoch [3/10], Step [200/235], Loss: 0.5229
Epoch [4/10], Step [100/235], Loss: 0.3901
Epoch [4/10], Step [200/235], Loss: 0.3713
Epoch [5/10], Step [100/235], Loss: 0.3010
Epoch [5/10], Step [200/235], Loss: 0.3885
Epoch [6/10], Step [100/235], Loss: 0.2924
Epoch [6/10], Step [200/235], Loss: 0.3261
Epoch [7/10], Step [100/235], Loss: 0.2697
Epoch [7/10], Step [200/235], Loss: 0.3028
Epoch [8/10], Step [100/235], Loss: 0.3758
Epoch [8/10], Step [200/235], Loss: 0.2732
Epoch [9/10], Step [100/235], Loss: 0.2874
Epoch [9/10], Step [200/235], Loss: 0.3152
Epoch [10/10], Step [100/235], Loss: 0.3307
Epoch [10/10], Step [200/235], Loss: 0.2918
✅ Model trained successfully
Test 

Converting to ONNX

In [29]:
import torch.onnx

def convert_to_onnx(model, onnx_file_path):
    dummy_input = torch.randn(1, input_size).to(device)
    torch.onnx.export(model, dummy_input, onnx_file_path,
                      export_params=True, opset_version=10, do_constant_folding=True)

    print(f"Model has been converted to ONNX and saved as {onnx_file_path}")

onnx_file_path = "mnist_model.onnx"
convert_to_onnx(model, onnx_file_path)

Model has been converted to ONNX and saved as mnist_model.onnx


Verify Inference

In [30]:
# This function is for the previous MNIST tutorial
def preprocess_image(image_path: str):
    """
    Preprocess an image for the MNIST model.

    Args:
        image_path (str): Path to the image file.

    Returns:
        np.ndarray: Preprocessed image.
    """

    # Load image, convert to grayscale, resize and normalize
    image = Image.open(image_path).convert('L')
    # Resize to match the input size of the model
    image = image.resize((14, 14))
    image = np.array(image).astype('float32') / 255
    image = image.reshape(1, 196)  # Reshape to (1, 196) for model input
    return image

In [31]:
from giza.agents.model import GizaModel

In [33]:
MODEL_ID = 651  # Update with your model ID
VERSION_ID = 2  # Update with your version ID

def prediction(image, model_id, version_id):
    model = GizaModel(id=model_id, version=version_id)

    (result, request_id) = model.predict(
        input_feed={"image": image}, verifiable=True
    )

    # Convert result to a PyTorch tensor
    result_tensor = torch.tensor(result)
    # Apply softmax to convert to probabilities
    temp = nn.Softmax(dim=1)
    probabilities = temp(result_tensor)
    # Use argmax to get the predicted class
    predicted_class = torch.argmax(probabilities, dim=1)

    return predicted_class.item(), request_id

def execution():
    image = preprocess_image("./imgs/zero.png")
    (result, request_id) = prediction(image, MODEL_ID, VERSION_ID)
    print("Result: ", result)
    print("Request id: ", request_id)

    return result, request_id


execution()

🚀 Starting deserialization process...
✅ Deserialization completed! 🎉
Result:  0
Request id:  3055227ba523412c90fddc1597733bc5


(0, '3055227ba523412c90fddc1597733bc5')